# A first look at the data

In [1]:
#get data from server
!wget --no-check-certificate https://www.keuper-labs.org/project_ss22.zip

--2022-05-30 09:00:00--  https://www.keuper-labs.org/project_ss22.zip
Resolving www.keuper-labs.org (www.keuper-labs.org)... 91.250.118.129
Connecting to www.keuper-labs.org (www.keuper-labs.org)|91.250.118.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 70859177 (68M) [application/zip]
Saving to: ‘project_ss22.zip.1’

project_ss22.zip.1  100%[===================>]  67,58M  9,15MB/s    in 7,6s    

2022-05-30 09:00:07 (8,84 MB/s) - ‘project_ss22.zip.1’ saved [70859177/70859177]



In [2]:
#unpack
!unzip project_ss22.zip

Archive:  project_ss22.zip
replace dressipi_recsys2022/candidate_items.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
import pandas as pd

In [4]:
items = pd.read_csv('dressipi_recsys2022/candidate_items.csv')
features = pd.read_csv('dressipi_recsys2022/item_features.csv')
train_purchases = pd.read_csv('dressipi_recsys2022/train_purchases.csv')
train_sessions = pd.read_csv('dressipi_recsys2022/train_sessions.csv')
test_leaderboard_sessions = pd.read_csv('dressipi_recsys2022/test_leaderboard_sessions.csv')

## have a first look

In [5]:
items.head()

,item_id
0,4
1,8
2,9
3,19
4,20


In [12]:
items.shape

(4990, 1)

In [6]:
features.head()

,item_id,feature_category_id,feature_value_id
0,2,56,365
1,2,62,801
2,2,68,351
3,2,33,802
4,2,72,75


In [11]:
features.shape

(471751, 3)

In [17]:
features['feature_category_id'].unique().size

73

In [28]:
features.groupby(['feature_category_id','feature_value_id']).size()

feature_category_id  feature_value_id
1                    60                      1
                     143                    10
                     358                     2
                     461                   924
                     517                     2
                                         ...  
72                   751                    55
                     829                   405
                     883                    65
73                   91                   2381
                     544                 10958
Length: 904, dtype: int64

In [35]:
#get all items-feature combination where the same feature has multiple values
df=features.groupby(['item_id','feature_category_id']).size().reset_index().rename(columns={0:'count'})
df[df['count']>1]

,item_id,feature_category_id,count
129,10,30,2
144,11,30,3
238,18,30,4
402,30,4,3
478,35,30,2
...,...,...,...
461080,28125,30,3
461095,28126,30,3
461191,28133,28,2
461193,28133,30,3


In [36]:
#all features tha have multiple assigments
df[df['count']>1]['feature_category_id'].unique()

array([30,  4, 46, 28, 53,  1])

In [7]:
train_purchases.head()

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114


In [13]:
train_purchases.shape

(1000000, 3)

In [18]:
train_purchases['item_id'].unique().size

18907

In [19]:
train_purchases['session_id'].unique().size

1000000

In [8]:
train_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [15]:
train_sessions.shape

(4743820, 3)

In [20]:
train_sessions['session_id'].unique().size

1000000

In [21]:
train_sessions['item_id'].unique().size

23496

In [9]:
test_leaderboard_sessions.head()

,session_id,item_id,date
0,26,19185,2021-06-16 09:53:54.158
1,200,17089,2021-06-25 12:23:40.811
2,200,17089,2021-06-25 12:24:36.631
3,200,8060,2021-06-25 12:24:41.677
4,200,4758,2021-06-25 12:24:50.692


In [16]:
test_leaderboard_sessions.shape

(229354, 3)

In [23]:
test_leaderboard_sessions['item_id'].unique().size

5647

In [24]:
test_leaderboard_sessions['session_id'].unique().size

50000

## Item Feature Vectors 

In [ ]:
# Features [30,  4, 46, 28, 53,  1] have multiple values at the same time -> use one-hot-encoding to fix this!
# -> https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [42]:
# combine all feture tuples to feature vector: WARNING works only AFTER one-hot-encoding or this subset with out multiple values per feature
df2=features[:100].pivot(index='item_id',columns='feature_category_id')
df2

feature_value_id                                           \
feature_category_id               3      4      5      7      11     15    16   
item_id                                                                         
2                                NaN    NaN    NaN  394.0    NaN    NaN  38.0   
3                              889.0  618.0  605.0  452.0  859.0    NaN   NaN   
4                              793.0  618.0  605.0  837.0    NaN    NaN   NaN   
7                                NaN    NaN    NaN  536.0    NaN  133.0   NaN   
8                              793.0  618.0  605.0  798.0  735.0    NaN   NaN   

                                          ...                              \
feature_category_id     17     18     19  ...     56     59     61     62   
item_id                                   ...                               
2                      NaN    NaN    NaN  ...  365.0    NaN  462.0  801.0   
3                    378.0  817.0  254.0  ...  365.0  180.0  706.0    NaN   
4                    378.0  289.0  148.0  ...  365.0  180.0  462.0    NaN   
7                      NaN    NaN   33.0  ...  153.0    NaN  706.0    NaN   
8                    378.0  817.0  700.0  ...  365.0  856.0  462.0    NaN   

                                                              
feature_category_id     63     65     68     69    72     73  
item_id                                                       
2                      NaN    NaN  351.0  885.0  75.0    NaN  
3                    861.0  521.0   14.0  592.0  75.0  544.0  
4                    861.0  521.0  373.0  538.0  75.0  544.0  
7                    816.0    NaN  739.0  592.0  75.0    NaN  
8                    861.0  521.0  351.0  592.0  75.0  544.0  

[5 rows x 33 columns]

In [41]:
df2.shape

(5, 33)